# 0. Introduction
Welcome to the Risk-Parity Portfolio Analysis project! In this project, you'll explore the concept of risk-parity, a popular portfolio management strategy that aims to allocate investments in a way that equalizes the risk contribution of each asset. This approach is widely used by hedge funds and institutional investors to create more balanced portfolios that are less sensitive to market volatility.

Throughout this project, you'll learn how to download financial data for various assets, calculate returns, and compute risk-parity weights using Python. By the end, you'll evaluate the performance of your risk-parity portfolio through key financial metrics such as annualized return, volatility, and Sharpe ratio. This hands-on experience will deepen your understanding of portfolio management and give you practical skills in financial data analysis. Whether you're new to finance or looking to enhance your quantitative finance skills, this project provides a solid foundation in risk-parity strategies and their application in real-world scenarios.

## Getting Started: Tips and Instructions

1. **Familiarize Yourself with the Notebook Structure:**
   - The notebook is organized into clearly defined sections, each focusing on a specific aspect of the risk-parity portfolio analysis. Take a moment to glance through the sections to understand the overall workflow.

2. **Review the Data:**
   - Since the data is already available, start by exploring the initial few cells to understand the data structure and what each column represents. This will give you context for the calculations and analysis you'll be performing.

3. **Run Cells Sequentially:**
   - Work through the notebook by executing each code cell in order. This will help you build your analysis step by step and ensure that each part of the project is functioning as expected.

4. **Experiment with Parameters:**
   - Feel free to tweak parameters such as the rolling window size for calculating risk-parity weights or adjust the assets included in the portfolio. Experimenting will help deepen your understanding of how these factors influence portfolio performance.

5. **Leverage the Plots:**
   - Use the generated plots to visually assess the impact of your calculations. The visualizations are crucial for understanding the risk-parity approach and how different assets contribute to the portfolio.

6. **Reflect:**
   - As you work through the notebook, consider why each step is necessary and how it contributes to the overall goal of building a risk-parity portfolio. If something isn’t clear, take a moment to reflect or explore additional resources.

Enjoy the process of exploring risk-parity strategies, and don't hesitate to dive deeper into the data or code to enhance your learning experience!

pip install requests-cache# 1. Import Libraries

In [41]:
!pip install --upgrade openbb-core yfinance

In [42]:
# Load necessary libraries
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import requests_cache
import time
from openbb import obb
obb.user.preferences.output_type = "dataframe"
# from ib_insync import *

# 2. Download Financial Data
We will download front-month futures data for S&P500, 10-year Treasuries, gold, and US dollar using the `yfinance` library.

In [43]:

data = obb.equity.price.historical("SPY", provider="yfinance", start="2023-01-01", end="2024-01-01")


1 Failed download:
['SPY']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


EmptyDataError: 
[Empty] -> No results found. Try adjusting the query parameters.

In [44]:
spy = yf.Ticker("SPY")
data = spy.history(period="1mo")  # Adjust period as needed
print(data)

YFRateLimitError: Too Many Requests. Rate limited. Try after a while.

In [25]:
ticker = yf.download(stocks)

[*********************100%***********************]  1 of 1 completed

1 Failed download:
['AMZN']: ReadTimeout(ReadTimeoutError("HTTPSConnectionPool(host='www.yahoo.com', port=443): Read timed out. (read timeout=30)"))


In [45]:
def get_spy_data():
    try:
        spy = yf.Ticker("SPY")
        data = spy.history(period="1mo")
        return data
    except Exception as e:
        print("Error:", e)
        return None

# Fetch data with delay
for _ in range(3):  # Try 3 times
    data = get_spy_data()
    if data is not None:
        break
    time.sleep(60)  # Wait 1 minute before retrying

Error: Too Many Requests. Rate limited. Try after a while.
Error: Too Many Requests. Rate limited. Try after a while.
Error: Too Many Requests. Rate limited. Try after a while.


In [46]:
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import yfinance as yf

def get_with_retry(ticker_symbol, max_retries=3, timeout=5):
    session = requests.Session()
    retry = Retry(connect=max_retries, backoff_factor=0.5)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    
    return yf.Ticker(ticker_symbol, session=session, timeout=timeout)

ticker = get_with_retry("AAPL")

TypeError: Ticker.__init__() got an unexpected keyword argument 'timeout'

In [47]:
futures_tickers = [
    "ES=F",   # E-mini S&P 500 Futures
    "NQ=F",   # E-mini Nasdaq-100 Futures
    "YM=F",   # E-mini Dow Jones Futures
    "RTY=F",  # E-mini Russell 2000 Futures
    "MES=F"   # Micro E-mini S&P 500 Futures
]




data = yf.download(futures_tickers)

[*********************100%***********************]  1 of 5 completed

5 Failed downloads:
['YM=F', 'MES=F', 'ES=F', 'NQ=F', 'RTY=F']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')
